In [ ]:
import nltk
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np
import datetime as dt 

In [640]:
df = pd.read_csv("./AccidentesAviones.csv")

# ETL


## Limpieza de los datos

In [ ]:
# Visualizamos el número de filas y columnas
df.shape

In [ ]:
# Visulizamos los tres primeros registros del dataset
df.head(3)

In [ ]:
# Visualizamos los valores faltantes o nulos
df.isna().sum()
# Al parecer los valores faltantes estan representados por el simbolo '?' por eso nos muestran ceros

In [641]:
#Procedamos a reemplazar ese simbolo '?' por valores nulos
df.replace('?',np.nan,inplace=True)

In [ ]:
# Ahora si podemos visualiazar los valores faltantes
df.isna().sum()

In [642]:
#Procedamos a reemplazar ese simbolo Nan por 'Sin datos' en las columnas alfanumericas
df[['Ruta','OperadOR','flight_no','route','ac_type','registration','cn_ln','summary']] = df[['Ruta','OperadOR','flight_no','route','ac_type',\
    'registration','cn_ln','summary']].fillna('Sin datos')
# Reemplazar ese simbolo Nan por cero en las columnas que deberían ser numericas
df[['all_aboard','PASAJEROS A BORDO', 'crew_aboard', 'cantidad de fallecidos', 'passenger_fatalities', \
    'crew_fatalities', 'ground']] = df[['all_aboard','PASAJEROS A BORDO', 'crew_aboard', 'cantidad de fallecidos', 'passenger_fatalities', \
    'crew_fatalities', 'ground']].fillna(0)

In [ ]:
# Visualizamos los últimos tres registros
df.tail(3)

Trabajaremos con el campo fecha

In [643]:
# Eliminamos la coma que separa el mes de los días en la fecha
df['fecha']=df.fecha.str.replace(',','')
#Creamos tres nuevas columnas para extraer el mes, día y año de la fecha
df[['mes','dia','año']]=df['fecha'].str.split(' ', expand=True)

In [644]:
# Cambiamos el nombre del mes por su número representativo
df['mes']=pd.to_datetime(df['mes'], format='%B').dt.month

In [645]:
# Hacemos la transformación de la columna fecha
df['fecha']=df['año'].astype(str)+'-'+df['mes'].astype(str)+'-'+df['dia'].astype(str)

In [646]:
# Se cambia el tipo objeto por tipo fecha y le cambiamos el formato a fecha
df['fecha'] = pd.to_datetime(df['fecha'], format='%Y-%m-%d', errors='coerce')

In [647]:
# Eliminamos columnas creadas para el trabajo de la fecha
df = df.drop(columns=['mes', 'dia', 'año'])

Trabajaremos ahora con el campo hora

In [648]:
# Visualizamos los registros que tengan mas de 4 caracteres
df[df['HORA declarada'].str.len()>4]

,Unnamed: 0,fecha,HORA declarada,Ruta,OperadOR,flight_no,route,ac_type,registration,cn_ln,all_aboard,PASAJEROS A BORDO,crew_aboard,cantidad de fallecidos,passenger_fatalities,crew_fatalities,ground,summary
84,84,1923-09-14,c 18:00,"Near Ivinghoe, Buckinghamshire, England",Daimler Airways,Sin datos,Croydon - Manchester,de Havilland DH-34,G-EBBS,29,5,3,2,5,3,2,0,Crashed while attempting a forced landing afte...
87,87,1923-12-23,c 02:30,Over the Mediterranean Sea,Military - French Navy,Sin datos,Toulon - Algiers,Zeppelin Dixmunde (airship),L-72,Sin datos,52,0,0,52,0,0,0,"Crashed while on a flight from Toulon, France ..."
95,95,1924-12-24,c 12:00,"Purley, England",Imperial Airways,Sin datos,London - Paris,de Havilland DH-34B,G-EBBX,36,8,7,1,8,7,1,0,"Shortly after taking off, witnesses observed a..."
117,117,1926-10-21,c 13:15,English Channel,Imperial Airways,Sin datos,Croydon - Paris,Handley Page W-10,G-EBMS,W10/3,12,10,2,0,0,0,0,The plane was on a flight from London to Paris...
132,132,1927-09-23,10:00,"Schleiz, Germany",Deutsche Lufthansa,Sin datos,Berlin - Munich,Dornier Merkur,D-585,075,6,4,2,6,4,2,0,From an altitude of 500 ft. the plane went out...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4380,4380,2002-02-22,02:30,"Off Dumaguete, Philippines",Military - U.S. Army,Sin datos,Basilan - Mactan Island,MH-47 Chinook helicopter,92-0471,M-3719,10,4,6,10,4,6,0,A helicopter ferrying U.S. troops in a counter...
4385,4385,2002-04-15,11:23,"Busan, South Korea",Air China,129,Beijing - Busan,Boeing B-767-200ER,B-2552,23308/127,166,155,11,128,120,8,0,"The aircraft crashed onto a mountain, in fog, ..."
4531,4531,2004-12-11,05:30,"Uberaba, Brazil",NHR Táxi-Aéreo,Sin datos,Sao Paulo - Uberaba,Embraer EMB-110 Bandeirante,PT-WAK,110071,2,0,2,2,0,2,1,The crew was performing a mail flight under To...
4742,4742,2009-08-26,0500Z,"Nganga Lingolo, Congo",Aero Fret Business,Sin datos,Pointe Noire - Brazzaville,Antonov An-12BK,TN-AIA,6344607,7,2,5,7,2,5,0,The cargo plane crashed 7 miles SE of Brazzavi...


In [649]:
# Se reemplaza el simbolo de ':' por espacio
df['HORA declarada']=df['HORA declarada'].str.replace(':','')

# Se reemplaza la letra de 'c' por espacio
df['HORA declarada']=df['HORA declarada'].str.replace('c','')

# Se reemplaza la letra 'z' por espacio
df['HORA declarada']=df['HORA declarada'].str.replace('Z','')

# Se reemplaza el simbolo de ';' por espacio
df['HORA declarada']=df['HORA declarada'].str.replace(';','')

# Se eliminan los espacios en blanco
df['HORA declarada']=df['HORA declarada'].str.strip()

#Creamos la columna Hora para reemplazar la anterior
df['HORA declarada']=pd.to_datetime(df['HORA declarada'], format='%H%M').dt.time

In [650]:
df[df['HORA declarada'].str.len()>4]

,Unnamed: 0,fecha,HORA declarada,Ruta,OperadOR,flight_no,route,ac_type,registration,cn_ln,all_aboard,PASAJEROS A BORDO,crew_aboard,cantidad de fallecidos,passenger_fatalities,crew_fatalities,ground,summary


In [651]:
#Procedemos a cambiar los tipos de valores considerados númericos
df[['all_aboard', 'PASAJEROS A BORDO', 'crew_aboard', 'cantidad de fallecidos', 'passenger_fatalities', 'crew_fatalities', \
    'ground']] = df[['all_aboard', 'PASAJEROS A BORDO', 'crew_aboard', 'cantidad de fallecidos', 'passenger_fatalities',\
        'crew_fatalities','ground']].astype('int64')

In [652]:
# Revisamos la cantidad de todo el personal abordo que tenga valor cero
df['all_aboard'][df['all_aboard']==0].count()

22

In [653]:
# Hacemos una suma entre los pasajeros a bordo y los tripulantes para reemplazar el valor cero.
df['all_aboard'] = np.where(df['all_aboard']==0, df['PASAJEROS A BORDO']+df['crew_aboard'], df['all_aboard'])

# Despues de realizar la suma de los pasajeros + los tripulantes si la suma dio cero, procedemos a sumar 
# la cantidad de pasajeros y tripulantes fallecidos para reemplazar los ceros del pesonal abordo.  
df['all_aboard'] = np.where(df['all_aboard']==0, df['passenger_fatalities']+df['crew_fatalities'], df['all_aboard']) 

In [654]:
# Revisamos nuevamente la cantidad de todo el personal abordo y notamos que se reemplazaron algunos.
df['all_aboard'][df['all_aboard']==0].count()

16

In [655]:
# Ahora con la información anterior procedemos a eliminar las filas que no tienen reflejado ningún pasajero o tripulante
df.drop(df.index[df.all_aboard==0], axis=0, inplace=True)

In [656]:
df.shape

(4992, 18)

Ya normalizadas las columna más significativas para nuetro proyecto, proseguimos a sacar información de la summary

In [657]:
#Función para extraer palabras claves sobre las causas del accidente

def iden_causa_acc(text):
    # Dividir el texto en palabras individuales
    tokens = word_tokenize(text.lower())
    
    # Definir las palabras clave y sus categorías correspondientes
    keywords = {
    'drogas': ['cocaine', 'drugs', 'narcotics', 'substance abuse', 'intoxication'],
    'falla mecanica': ['failure', 'mechanical', 'breakdown', 'malfunction', 'defect'],
    'error humano': ['error', 'mistake', 'negligence', 'inattention', 'fatigue'],
    'condiciones climaticas': ['weather', 'storm', 'fog', 'visibility', 'wind','storm','thunderstorm', 'fog', 'heavy rain', \
        'strong winds', 'hail', 'ice', 'snow', 'severe turbulence', 'reduced visibility'],
    'error del piloto': ['pilot', 'crew', 'aviator', 'maneuvering', 'navigation'],
    'problemas de mantenimiento': ['maintenance', 'inspection', 'repair', 'component', 'service'],
    'terrorismo': ['terrorism','shot down in flames' 'sabotage', 'explosion', 'hijacking', 'security breach'],
    'falla estructural': ['structural', 'collapse', 'material', 'design', 'construction'],
    'fuego': ['exploded','fire', 'smoke', 'combustion', 'burn', 'ignition',"engine fire", "cabin fire", "fuel fire", \
        "electrical fire", "wing fire", "cargo fire", "cockpit fire", "landing gear fire", "brake fire", "tail fire", \
            "apu fire", "battery fire", "smoke in the cockpit"],
    'choques con aves': ['bird', 'collision', 'wildlife', 'feather', 'impact'],
    'control de tráfico aéreo': ['air traffic control', 'communication', 'coordination', 'instructions', 'confusion'],
    'incursión en la pista': ['runway', 'incursion', 'taxiing', 'conflict', 'intersection'],
    'problemas de combustible': ['fuel', 'leakage', 'exhaustion', 'contamination', 'shortage'],
    'colisión en el aire': ['midair', 'collision', 'encounter', 'aircraft', 'traffic'],
    'falla de instrumento': ['instrument', 'failure', 'malfunction', 'calibration', 'display'],
    'falla del motor': ['engine', 'failure', 'malfunction', 'stall', 'power loss'],
    'vuelo controlado hacia el terreno': ['controlled flight into terrain', 'CFIT', 'terrain', 'obstacle', 'crash'],
    'pérdida de control': ['loss of control', 'LOC', 'unstable', 'maneuver', 'stability'],
    'cortante de viento': ['windshear', 'microburst', 'wind gradient', 'sudden', 'downburst'],
    'formación de hielo': ['icing', 'ice', 'frost', 'de-icing', 'anti-icing'],
    'desconocido': ['unknown', 'unclear', 'unidentified', 'mysterious', 'puzzling'],
    'actividad inusual': ['unusual activity', 'strange', 'abnormal', 'bizarre', 'peculiar'],
    'eventos cósmicos': ['cosmic events', 'space', 'meteor', 'asteroid', 'celestial'],
    'sobrenatural': ['supernatural', 'paranormal', 'mystical', 'ghost', 'curse']
}
    
    # Comprobar si hay coincidencias de palabras clave en el texto.
    for category, words in keywords.items():
        if any(word in tokens for word in words):
            return category
    
    # Si no se encontraron coincidencias, devuelva una categoría predeterminada
    return 'otros'



In [658]:
df['causa_accidente'] = df['summary'].apply(iden_causa_acc)

valores_causa_accidente = df['causa_accidente'].unique()

otras_filas = df[df['causa_accidente'] == 'otros']

In [659]:
# Accidentes no identificados
otras_filas

,Unnamed: 0,fecha,HORA declarada,Ruta,OperadOR,flight_no,route,ac_type,registration,cn_ln,all_aboard,PASAJEROS A BORDO,crew_aboard,cantidad de fallecidos,passenger_fatalities,crew_fatalities,ground,summary,causa_accidente
10,10,1916-10-01,23:45:00,"Potters Bar, England",Military - German Navy,Sin datos,Sin datos,Zeppelin L-31 (airship),Sin datos,Sin datos,19,0,19,19,0,19,0,Shot down in flames by the British 39th Home D...,otros
14,14,1917-03-30,NaT,Off Northern Germany,Military - German Navy,Sin datos,Sin datos,Schutte-Lanz S-L-9 (airship),Sin datos,Sin datos,23,0,0,23,0,0,0,Struck by lightning and crashed into the Balti...,otros
27,27,1919-08-02,NaT,"Verona, Italy",Caproni Company,Sin datos,Venice Taliedo,Caproni Ca.48,Sin datos,Sin datos,14,12,2,14,12,2,0,As the plane was passing over Verona the wings...,otros
28,28,1919-10-02,NaT,"Newcastle, England",Aircraft Transport and Travel,Sin datos,Sin datos,De Havilland DH-4,Sin datos,Sin datos,1,0,1,1,0,1,0,Sin datos,otros
32,32,1919-12-11,NaT,"Catherham, Surrey, UK",Aircraft Travel Transport,Sin datos,Paris - Hounslow,de Havilland DH-4,G-EAHF,Sin datos,1,1,1,1,1,1,0,Crashed in a field while attemptting to land.,otros
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4969,4969,2018-04-11,08:00:00,"Boufarik AB, Algeria",Military - Algerian Air Force,Sin datos,Boufarik AB - Bechar - Tindouf,Ilyushin 76-TD,7T-WIP,1043419636,257,247,10,257,247,10,0,The Algerian military plane crashed soon after...,otros
4976,4976,2018-09-17,22:07:00,"Near Latakia-Khmeimim Air Base, Iran",Military - Russian Air Force,Sin datos,Training,Ilyushin Il-20M,RF-93610,173011504,15,0,15,15,0,15,0,Crashed into the sea. Hit by a surface-to-air ...,otros
4987,4987,2019-06-03,13:00:00,"Near Lipo, India",Military - Indian Air Force,Sin datos,Jorhat-Rowriah - Mechuka,Antonov An-32,K2752,1009,13,5,8,13,5,8,0,Crashed about 34km WNW of Mechuka.,otros
4991,4991,2019-11-24,09:15:00,"Goma, Democratic Republic of Congo",Busy Bee Congo,Sin datos,Goma - Beni,Dornier 228-201,9S-GNH,8030,19,17,2,19,17,2,7,The passenger plane crashed about a minute aft...,otros


In [660]:
# Función para definir las aeronaves

def iden_aeronave(text):
    # Dividir el texto en palabras individuales
    tokens = word_tokenize(text.lower())
    
    # Definir las palabras clave y sus categorías correspondientes
    keywords = {
    'avión': ['plane', 'airplane', 'ship', 'martin', 'aeroplane'],
    'helicoptero': ['helicopter'],
    'dirigible': ['dirigible','blimp'],
    'aeronaves': ['airship', 'aircraft']
    }
    
    # Comprobar si hay coincidencias de palabras clave en el texto.
    for category, words in keywords.items():
        if any(word in tokens for word in words):
            return category
    
    # Si no se encontraron coincidencias, devuelva una categoría predeterminada
    return 'Si definir'


In [662]:
df['aeronaves'] = df['summary'].apply(iden_aeronave)

valores_aeronave = df['aeronaves'].unique()

otras_aeronave = df[df['aeronaves'] == 'otros']

In [664]:
# Eliminamos la columna 'Unnamed: 0' por representar solo un correlativo, nos quedamos con el resto para probar el EDA
df = df.drop(columns=['Unnamed: 0'])

In [665]:
# Cambiamos el nombre a las columnas para que sean más entendible a la hora de analizar
df = df.rename(columns={'HORA declarada': 'hora', 'Ruta': 'salida', 'OperadOR': 'operador', 'flight_no':'numero_vuelo', \
    'route':'destino', 'ac_type':'tipo_aeronave', 'registration': 'registro', 'cn_ln': 'contro_interno', \
    'all_aboard':'personal_total', 'PASAJEROS A BORDO':'pasajeros_a_bordo', 'crew_aboard':'tripulantes_a_bordo', \
    'cantidad de fallecidos':'cantidad_fallecidos', 'passenger_fatalities':'pasajeros_fallecidos', \
    'crew_fatalities':'tripulantes_fallecidos', 'ground':'fallecidos_en_tierra', 'summary':'resumen'})

In [666]:
df.head()

,fecha,hora,salida,operador,numero_vuelo,destino,tipo_aeronave,registro,contro_interno,personal_total,pasajeros_a_bordo,tripulantes_a_bordo,cantidad_fallecidos,pasajeros_fallecidos,tripulantes_fallecidos,fallecidos_en_tierra,resumen,causa_accidente,aeronaves
0,1908-09-17,17:18:00,"Fort Myer, Virginia",Military - U.S. Army,Sin datos,Demonstration,Wright Flyer III,Sin datos,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly...",colisión en el aire,avión
1,1909-09-07,NaT,"Juvisy-sur-Orge, France",Sin datos,Sin datos,Air show,Wright Byplane,SC1,Sin datos,1,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...,error del piloto,Si definir
2,1912-07-12,06:30:00,"Atlantic City, New Jersey",Military - U.S. Navy,Sin datos,Test flight,Dirigible,Sin datos,Sin datos,5,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...,fuego,dirigible
3,1913-08-06,NaT,"Victoria, British Columbia, Canada",Private,Sin datos,Sin datos,Curtiss seaplane,Sin datos,Sin datos,1,0,1,1,0,1,0,The first fatal airplane accident in Canada oc...,error del piloto,avión
4,1913-09-09,18:30:00,Over the North Sea,Military - German Navy,Sin datos,Sin datos,Zeppelin L-1 (airship),Sin datos,Sin datos,20,0,0,14,0,0,0,The airship flew into a thunderstorm and encou...,condiciones climaticas,avión


In [667]:
df.reset_index().to_csv('AccidentesAvionesModificado.csv', header = True, index=False)